In [ ]:
!pip install --quiet mrjob==0.7.4

In [ ]:
#mount my google drive
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


**Ex2**

In [ ]:
from mrjob.job import MRJob

def mapper(self, key, value, context):
        """ generated source for method map """
        fields = value.__str__().split(":")
        fields[0] = fields[0].trim()
        pattern = fields[0].substring(1, len(fields[0]) - 1)
        count = Long.parseLong(fields[1].trim())
        items = StringTokenizer(pattern, ",")
        while items.hasMoreTokens():
            self.newKey.set(item)
            self.newValue.set("[" + pattern.trim() + "]" + ":" + count)
            context.write(self.newKey, self.newValue)

In [ ]:
%%file Bjoin.py
from mrjob.job import MRJob

class Bjoin(MRJob):
    def mapper_init(self):
        f = open("/content/gdrive/MyDrive/datasets/mr/depc.csv").read()
        self.empc = f.split("\n")
        self.new_list = []
        for i in self.empc:
            self.new_list.append(list(map(str,i.split(","))))
        self.hash_map = {}
        for i in range(len(self.new_list)):
            if self.new_list[i] != [""]:
                if self.new_list[i][0] in self.hash_map:
                    self.hash_map[self.new_list[i][0]].append(self.new_list[i])
                else:
                    self.hash_map[self.new_list[i][0]] = [self.new_list[i]]

    def mapper(self, key, line):
        request = line.split(",")
        dno = request[6]
        if dno in self.hash_map.keys():
            for values in self.hash_map[dno]:
                yield (request,values)

if __name__ == '__main__':
    Bjoin.run()

Overwriting Bjoin.py


In [ ]:
!python3 Bjoin.py "/content/gdrive/MyDrive/datasets/mr/empc.csv"

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/Bjoin.root.20230920.095536.533798
Running step 1 of 1...
job output is in /tmp/Bjoin.root.20230920.095536.533798/output
Streaming final output from /tmp/Bjoin.root.20230920.095536.533798/output...
["108", "Alicia", "1958-07-19", "F", "25000", "106", "4"]	["4", "Administration", "106", "1985-01-01"]
["101", "John", "1955-01-09", "M", "30000", "102", "5"]	["5", "Research", "102", "1978-05-22"]
["102", "Franklin", "1945-12-08", "M", "40000", "105", "5"]	["5", "Research", "102", "1978-05-22"]
["103", "Joyce", "1962-07-31", "F", "25000", "102", "5"]	["5", "Research", "102", "1978-05-22"]
["104", "Ramesh", "1952-09-15", "M", "38000", "102", "5"]	["5", "Research", "102", "1978-05-22"]
["105", "James", "2027-11-10", "M", "55000", "", "1"]	["1", "Headquater", "105", "1971-06-19"]
["106", "Jennifer", "1931-06-20", "F", "43000", "105", "4"]	["4", "Administration", "106", "1985-

**Ex3**(assuming a month to be 31 days)

---



In [ ]:
%%file monavg.py
from mrjob.job import MRJob
import re
import shlex

class MonAvg(MRJob):

    def mapper(self, key, line):

      if "Row" not in line:
         record = shlex.shlex(line,posix=True)
         record.whitespace += ','
         record.whitespace_split = True
         record = list(record)
         date = re.split('/',record[3])
         firsttime = record[6].replace(',','')
         reptime = record[7].replace(',','')
         if(int(date[1])<10):
            record[3] = "0"+date[1]+"/"+date[0]+"/"+date[2]
         else:
          record[3] = date[1]+"/"+date[0]+"/"+date[2]
         yield date[0]+" "+date[2], [record[3],firsttime,reptime]

    def reducer(self, eno , lis):
       lis = list(lis)
       lis.sort()
       avgfirst,avgrep = 0.0,0.0
       count = 0
       for i in lis:
        count += 1
        avgfirst = avgfirst + int(i[1])
        avgrep = avgrep + int(i[2])
        yield i[0] , [avgfirst/count,avgrep/count]

if __name__ == '__main__':
     MonAvg.run()

Overwriting monavg.py


In [ ]:
!python monavg.py "/content/gdrive/My Drive/datasets/daily-website-visitors.csv"

No configs found; falling back on auto-configuration
No configs specified for inline runner
Creating temp directory /tmp/monavg.root.20230920.095544.511496
Running step 1 of 1...
job output is in /tmp/monavg.root.20230920.095544.511496/output
Streaming final output from /tmp/monavg.root.20230920.095544.511496/output...
"01/7/2016"	[1730.0, 496.0]
"02/7/2016"	[1272.0, 360.5]
"03/7/2016"	[1190.6666666666667, 333.3333333333333]
"04/7/2016"	[1306.25, 372.75]
"05/7/2016"	[1492.0, 419.2]
"06/7/2016"	[1618.0, 444.8333333333333]
"07/7/2016"	[1693.7142857142858, 460.14285714285717]
"08/7/2016"	[1718.375, 463.875]
"09/7/2016"	[1642.5555555555557, 442.22222222222223]
"10/7/2016"	[1610.4, 431.4]
"11/7/2016"	[1664.7272727272727, 451.0]
"12/7/2016"	[1722.0, 464.0833333333333]
"13/7/2016"	[1787.8461538461538, 475.38461538461536]
"14/7/2016"	[1818.2857142857142, 484.2857142857143]
"15/7/2016"	[1816.8, 487.26666666666665]
"16/7/2016"	[1762.6875, 473.75]
"17/7/2016"	[1736.5882352941176, 464.529411764705